# DeepInfra

[DeepInfra](https://deepinfra.com/?utm_source=langchain) 是一个无服务器的推理即服务平台，提供对[各种 LLM](https://deepinfra.com/models?utm_source=langchain) 和[Embedding 模型](https://deepinfra.com/models?type=embeddings&utm_source=langchain)的访问。本笔记介绍了如何在 LangChain 中使用 DeepInfra 进行聊天模型。

## 设置环境变量 API 密钥
请务必从 DeepInfra 获取您的 API 密钥。您需要[登录](https://deepinfra.com/login?from=%2Fdash)并获取一个新的令牌。

您将获得 1 小时的免费无服务器 GPU 计算时间来测试不同的模型。（参见[此处](https://github.com/deepinfra/deepctl#deepctl)）
您可以使用 `deepctl auth token` 命令打印您的令牌。

In [ ]:
# get a new token: https://deepinfra.com/login?from=%2Fdash

import os
from getpass import getpass

from langchain_community.chat_models import ChatDeepInfra
from langchain_core.messages import HumanMessage

DEEPINFRA_API_TOKEN = getpass()

# or pass deepinfra_api_token parameter to the ChatDeepInfra constructor
os.environ["DEEPINFRA_API_TOKEN"] = DEEPINFRA_API_TOKEN

chat = ChatDeepInfra(model="meta-llama/Llama-2-7b-chat-hf")

messages = [
    HumanMessage(
        content="Translate this sentence from English to French. I love programming."
    )
]
chat.invoke(messages)

## `ChatDeepInfra` 也支持异步和流式处理功能：

In [ ]:
from langchain_core.callbacks import StreamingStdOutCallbackHandler

In [ ]:
await chat.agenerate([messages])

In [ ]:
chat = ChatDeepInfra(
    streaming=True,
    verbose=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)
chat.invoke(messages)

# 工具调用

DeepInfra 目前仅支持 invoke 和 async invoke 工具调用。

有关支持工具调用的模型的完整列表，请参阅我们的[工具调用文档](https://deepinfra.com/docs/advanced/function_calling)。

In [ ]:
import asyncio

from dotenv import find_dotenv, load_dotenv
from langchain_community.chat_models import ChatDeepInfra
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from pydantic import BaseModel

model_name = "meta-llama/Meta-Llama-3-70B-Instruct"

_ = load_dotenv(find_dotenv())


# Langchain tool
@tool
def foo(something):
    """
    Called when foo
    """
    pass


# Pydantic class
class Bar(BaseModel):
    """
    Called when Bar
    """

    pass


llm = ChatDeepInfra(model=model_name)
tools = [foo, Bar]
llm_with_tools = llm.bind_tools(tools)
messages = [
    HumanMessage("Foo and bar, please."),
]

response = llm_with_tools.invoke(messages)
print(response.tool_calls)
# [{'name': 'foo', 'args': {'something': None}, 'id': 'call_Mi4N4wAtW89OlbizFE1aDxDj'}, {'name': 'Bar', 'args': {}, 'id': 'call_daiE0mW454j2O1KVbmET4s2r'}]


async def call_ainvoke():
    result = await llm_with_tools.ainvoke(messages)
    print(result.tool_calls)


# Async call
asyncio.run(call_ainvoke())
# [{'name': 'foo', 'args': {'something': None}, 'id': 'call_ZH7FetmgSot4LHcMU6CEb8tI'}, {'name': 'Bar', 'args': {}, 'id': 'call_2MQhDifAJVoijZEvH8PeFSVB'}]